<a href="https://colab.research.google.com/github/rituparna50/MarsPosition/blob/main/Near_side_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using HMI near side images to validate MarsCam near side images.
We have already figured out a way to identify dates when the opening angle is between 90-270 degrees. But that is for far-side. In order to validate the utilisation of the MarsCam images, we first need to do near-side validation.

In [6]:
import astropy
import numpy as np

In [4]:
from astropy.time import Time
from astropy import units as u

In [10]:
DateTime = "2023-11-01 09:00:00"
when = Time(DateTime)
print(when)

2023-11-01 09:00:00.000


# Difference between Astropy and Skyfield libraries, even though both are Python packages

Astropy - Designed to handle a wide range of astronomical tasks from time and coordinate trasnformations to unit conversions, data visualization, cosmological calcualtions etc. Aims to provide a one-stop solution for many of the routine tasks in astronomy and astrophysics. Large and complex as it has a wide range of functionalities.

Skyfield - More focsed. Primary goal is to provide accurate and high-performance tools for computing the *positions of celestial bodies*. Much more high precision in terms of speed when computing positions using the JPL ephemerides.



In [ ]:
pip install skyfield datetime

# Skyfield and datetime are two different python packages we will need.
# Could not write as
    # pip install skyfield
    # pip install datetime

  # Have to write as -- pip install skyfield datetime --> Together